## 저장된 모델 불러오기

In [1]:
import torch
import os
import torch.nn.functional as F
import matplotlib.pyplot as plt
import matplotlib
from torchvision.utils import make_grid, save_image
import PIL
import torch
import torchvision.models as models
import torch.nn as nn
import cv2
import torchvision
import torchvision.transforms as transforms
import datetime
import numpy as np

sample = torch.load(os.path.join(os.getcwd(), 'resnet18_weight/checkpoint/ckpt.pth'))

In [2]:
resnet18 = models.resnet18(pretrained = False)

resnet18.fc = nn.Linear(512, 2)
print(resnet18.fc)

Linear(in_features=512, out_features=2, bias=True)


In [3]:
resnet18.load_state_dict(sample['net'])

<All keys matched successfully>

## CAM 만들기

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.4876, 0.4544, 0.4165), (0.2257, 0.2209, 0.2212)),
])

test_dataset = torchvision.datasets.ImageFolder(root = os.path.join(os.getcwd(), 'CatvsDog_test'), transform = transform_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 1, shuffle = True, num_workers = 0)


current_time = datetime.datetime.now() + datetime.timedelta(hours= 9)
current_time = current_time.strftime('%Y-%m-%d-%H:%M')


saved_loc = os.path.join('./', current_time)
if os.path.exists(saved_loc):
    shutil.rmtree(saved_loc)
os.mkdir(saved_loc)

print("결과 저장 위치: ", saved_loc)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

결과 저장 위치:  ./2021-05-14-23:05


In [5]:
resnet18.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [6]:
# final conv layer name 
finalconv_name = 'layer4'

# inference mode
resnet18.eval()

# number of result
num_result = 5


feature_blobs = []
backward_feature = []

# output으로 나오는 feature를 feature_blobs에 append하도록
def hook_feature(module, input, output):
    feature_blobs.append(output.cpu().data.numpy())
    
# Grad-CAM
def backward_hook(module, input, output):
    backward_feature.append(output[0])

    
resnet18._modules.get(finalconv_name).register_forward_hook(hook_feature)
resnet18._modules.get(finalconv_name).register_backward_hook(backward_hook)


# get the softmax weight
params = list(resnet18.parameters())
weight_softmax = np.squeeze(params[-2].cpu().detach().numpy()) # [2, 512]


# generate the class activation maps
def returnCAM(feature_conv, weight_softmax, class_idx):
    size_upsample = (224, 224)
    _, nc, h, w = feature_conv.shape # nc : number of channel, h: height, w: width
    output_cam = []
    # weight 중에서 class index에 해당하는 것만 뽑은 다음, 이를 conv feature와 곱연산
    cam = weight_softmax[class_idx].dot(feature_conv.reshape((nc, h*w))) 
    cam = cam.reshape(h, w)
    cam = cam - np.min(cam)
    cam_img = cam / np.max(cam)
    cam_img = np.uint8(255 * cam_img)
    output_cam.append(cv2.resize(cam_img, size_upsample))
    return output_cam


for i, (image, target) in enumerate(test_loader):
    
    # 모델의 input으로 주기 위한 image는 따로 설정
    image_for_model = image.clone().detach()

    # Image denormalization, using mean and std that i was used.
    image[0][0] *= 0.2257
    image[0][1] *= 0.2209
    image[0][2] *= 0.2212
    
    image[0][0] += 0.4876
    image[0][1] += 0.4544
    image[0][2] += 0.4165
    

    # 모델의 input으로 사용하도록.
    image_tensor = image_for_model.to(device)
    logit = resnet18(image_tensor)
    h_x = F.softmax(logit, dim=1).data.squeeze()
    
    probs, idx = h_x.sort(0, True)
    print("True label : %d, Predicted label : %d, Probability : %.2f" % (target.item(), idx[0].item(), probs[0].item()))
    
    
    # ============================= #
    # ==== Grad-CAM main lines ==== #
    # ============================= #
    
    score = logit[:, idx[0]].squeeze() # 예측값 y^c
    score.backward(retain_graph = True) # 예측값 y^c에 대해서 backprop 진행
    
    activations = torch.Tensor(feature_blobs[0]).to(device) # (1, 512, 7, 7), forward activations
    gradients = backward_feature[0] # (1, 512, 7, 7), backward gradients
    b, k, u, v = gradients.size()
    
    alpha = gradients.view(b, k, -1).mean(2) # (1, 512, 7*7) => (1, 512), feature map k의 'importance'
    weights = alpha.view(b, k, 1, 1) # (1, 512, 1, 1)
    
    grad_cam_map = (weights*activations).sum(1, keepdim = True) # alpha * A^k = (1, 512, 7, 7) => (1, 1, 7, 7)
    grad_cam_map = F.relu(grad_cam_map) # Apply R e L U
    grad_cam_map = F.interpolate(grad_cam_map, size=(224, 224), mode='bilinear', align_corners=False) # (1, 1, 224, 224)
    map_min, map_max = grad_cam_map.min(), grad_cam_map.max()
    grad_cam_map = (grad_cam_map - map_min).div(map_max - map_min).data # (1, 1, 224, 224), min-max scaling

    # grad_cam_map.squeeze() : (224, 224)
    grad_heatmap = cv2.applyColorMap(np.uint8(255 * grad_cam_map.squeeze().cpu()), cv2.COLORMAP_JET) # (224, 224, 3), numpy 
    grad_heatmap = torch.from_numpy(grad_heatmap).permute(2, 0, 1).float().div(255) # (3, 224, 224)
    b, g, r = grad_heatmap.split(1)
    grad_heatmap = torch.cat([r, g, b]) # (3, 244, 244), opencv's default format is BGR, so we need to change it as RGB format.

    save_image(grad_heatmap, os.path.join(saved_loc, "Grad_CAM_%d.jpg" % (i+1)))
    
    grad_result = grad_heatmap + image.cpu() # (1, 3, 224, 224)
    grad_result = grad_result.div(grad_result.max()).squeeze() # (3, 224, 224)
    
    save_image(grad_result, os.path.join(saved_loc, "GradCAM&image_%d.jpg" % (i+1)))
    
    
    image_list = []
    
    image_list.append(torch.stack([image.squeeze().cpu(), grad_heatmap, grad_result], 0)) # (3, 3, 224, 224)
    
    images = make_grid(torch.cat(image_list, 0), nrow = 3)
    
    save_image(images, os.path.join(saved_loc, "Final_Result_%d.jpg" % (i+1)))
    

    if i + 1 == num_result:
        break
        
    feature_blobs.clear()
    backward_feature.clear()

feature_blobs.clear()
backward_feature.clear()

True label : 1, Predicted label : 1, Probability : 1.00
True label : 0, Predicted label : 0, Probability : 1.00
True label : 0, Predicted label : 0, Probability : 1.00
True label : 0, Predicted label : 0, Probability : 1.00
True label : 0, Predicted label : 0, Probability : 1.00
